In [138]:
# Notification 
# 
import requests
def telegram_message(msg):
    token = "6793951865:AAEbF021k459kjqKFDhpBynotlKwoocikPw"
    url = f"https://api.telegram.org/bot{token}"
    params = {"chat_id": "5043550756", "text":msg }
    r = requests.get(url + "/sendMessage", params=params)
    return f"done {r}"


In [139]:
#optional
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Energy Market Data

In [6]:
#define ENTSOE Api Key 
#(token request(API key) via https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_authentication_and_authorisation see "2. Authentication and Authorisation")
API_KEY = "55c76530-b737-48af-bf6e-40d4ebde04f2"

In [1]:
#import relevant packages
from entsoe import EntsoePandasClient
import pandas as pd
import datetime as dt

In [154]:
# Entsoe API Call to retrieve data
def get_TSO_generation_data(geozone, start, end):
    client = EntsoePandasClient(api_key=API_KEY)
    start = pd.Timestamp(start, tz='Europe/Berlin')
    end = pd.Timestamp(end, tz='Europe/Berlin')
    return client.query_generation(geozone, start=start,end=end, psr_type=None)

# Transformation to hourly and erasure of consumption columns    
def cleanup_generation_data(df_to_be_cleaned):
    # following columns also consume power (Hydro, solar, wind Onshore)
    drop_consumption = df_to_be_cleaned.xs('Actual Consumption', level=1, axis=1,drop_level=False).columns
    df_to_be_cleaned.xs('Actual Consumption', level=1, axis=1,drop_level=False).columns
    # therefore get rid of consumption for the fact that consumption does not produce co2
    df_generation_in_MW=df_to_be_cleaned.drop(columns = drop_consumption.tolist())
    #drop header level 1
    df_generation_in_MW.columns= df_generation_in_MW.columns.droplevel(1)
    df_generation_in_MW.head(5)
    #df_generation_in_MW.resample('H', closed='right').sum().reset_index().head(5)
    df_generation_in_MWh = df_generation_in_MW.resample('h').sum()#.reset_index()
    return df_generation_in_MWh
    ##df_generation_in_MW.to_csv("GenerationDE.csv")



### Download generation data
Duration: about 3 hours

In [400]:
#mappings https://github.com/EnergieID/entsoe-py/blob/079fcd5ce301e2394efa613ea14eb96edc2a77aa/entsoe/mappings.py#L140

country_code50 = 'DE_50HZ'  
country_codeAMP = 'DE_AMPRION' 
country_codeTRA = 'DE_TRANSNET' 
country_codeTEN = 'DE_TENNET' 


start = dt.datetime(2015,1,1)
end = dt.datetime(2024,3,1)

telegram_message("Los geht es 2015 - 2024")
try:
    df_50HZ     = cleanup_generation_data(get_TSO_generation_data(country_code50, start, end))
    telegram_message("done: df_50HZ done 2015 - 2024")
except:
    telegram_message("error:df_50HZ done 2015 - 2024")
    

try:
    df_AMPRION  = cleanup_generation_data(get_TSO_generation_data(country_codeAMP, start, end))
    telegram_message("done: df_AMPRION done 2015 - 2024")
except:
    telegram_message("error: df_AMPRION done 2015 - 2024")
    
try:    
    df_TRANSNET = cleanup_generation_data(get_TSO_generation_data(country_codeTRA, start, end))
    telegram_message("done: df_TRANSNET done 2015 - 2024")
except:
    telegram_message("error: df_TRANSNET done 2015 - 2024")

try:
    df_TENNET   = cleanup_generation_data(get_TSO_generation_data(country_codeTEN, start, end))
    telegram_message("done: df_TENNET done 2015 - 2024")
except:
    telegram_message("error: df_TENNET done 2015 - 2024")

#Export data sets to file
df_50HZ.to_parquet("data/2015-2024_df_50HZ.parquet")
df_AMPRION.to_parquet("data/2015-2024_df_AMPRION.parquet")
df_TRANSNET.to_parquet("data/2015-2024_df_TRANSNET.parquet")
df_TENNET.to_parquet("data/2015-2024_df_TENNET.parquet")



KeyboardInterrupt: 

### Download day-ahead price data

Entso-E Api provides day-ahead prices for german geozone:
- DE_AT_LU from 2015-01-05 to 2018-09-30
- DE_LU from 2018-10-01 to now

In [156]:
def get_day_ahead_price(geozone, start, end):
    client = EntsoePandasClient(api_key=API_KEY)
    start = pd.Timestamp(start, tz='Europe/Berlin')
    end = pd.Timestamp(end, tz='Europe/Berlin')
    return client.query_day_ahead_prices(geozone, start=start,end=end)

In [170]:
client = EntsoePandasClient(api_key=API_KEY)
start_dap = pd.Timestamp('20150101', tz='Europe/Berlin')
end_dap = pd.Timestamp('20240301', tz='Europe/Berlin')

dap_DE_AT_LU = client.query_day_ahead_prices('DE_AT_LU', start=start_dap, end=end_dap)
dap_DE_LU = client.query_day_ahead_prices('DE_LU', start=start_dap, end=end_dap)

### Merge dap_DE_LU & dap_DE_AT_LU and save as file

In [171]:
dap_DE = pd.concat([dap_DE_AT_LU, dap_DE_LU])
dap_DE.to_pickle("data/2015-2024_DAP_DE_Series.pkl")
dap_DE

2015-01-05 00:00:00+01:00    22.34
2015-01-05 01:00:00+01:00    17.93
2015-01-05 02:00:00+01:00    15.17
2015-01-05 03:00:00+01:00    16.38
2015-01-05 04:00:00+01:00    17.38
                             ...  
2024-02-29 20:00:00+01:00    72.77
2024-02-29 21:00:00+01:00    67.25
2024-02-29 22:00:00+01:00    64.90
2024-02-29 23:00:00+01:00    60.90
2024-03-01 00:00:00+01:00    62.04
Freq: 60min, Length: 80233, dtype: float64

# Weather data

##### Stations of each TSO zone
| Zone | Station Name | station_id obs | station_id forecast | lat | lon | height |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
| DE_50Hertz   | Potsdam         | 3987| 10379 | 52.23  | 13.04  |  99
| DE_AMPRION   | Saarbruecken  | 4336 | 10708 | 49.21  |  7.11 |  320 |
| DE_TENNET    | Nuremberg          | 3668 | 10763 | 49.50  |  11.05  |  314 |
| DE_TRANSNET  | Konstanz        | 2712 | 10929 | 47.69  |  9.13  |  428 |


In [12]:
#import relevant packages
from wetterdienst.provider.dwd.observation import (DwdObservationRequest, 
                                                   DwdObservationPeriod, 
                                                   DwdObservationResolution, 
                                                   DwdObservationParameter, 
                                                   DwdObservationDataset) # =Observation data (historical data)

In [13]:
#define weather params
request_parameter = [
 'radiation_global',
 'sunshine_duration',
 'temperature_air_mean_200',
 'temperature_dew_point_mean_200',
 'wind_direction',
 'wind_speed']

### Call weather data

In [14]:
start_weather = dt.datetime(2014,12,31)
end_weather = dt.datetime(2024,3,2)

In [16]:
# DWD Request
def dwd_request(request_parameter, start, end, station_id):
    request = DwdObservationRequest(
        parameter=request_parameter,
        resolution=DwdObservationResolution.HOURLY,
        start_date=start_weather,
        end_date=end_weather,
    ).filter_by_station_id(station_id=[station_id])
    return request.values.all().df.to_pandas()

# function to loop every weather param in one DWD request
def concatenate_df(station_id, start_weather, end_weather):
    df_list=[]
    for req in request_parameter:
        print(req)
        egal = dwd_request(req,start_weather,end_weather,station_id)#.head(1))
        egal = egal.set_index("date")
        egal = egal.rename(columns={"value":req})
        egal = egal.drop(columns=["station_id","dataset","parameter","quality"])
        df_list.append(egal)
    return pd.concat(df_list, axis=1)

In [18]:
start_weather = dt.datetime(2014,12,31)
end_weather = dt.datetime(2024,3,2)

# call weather data
weather_hist_DE_50Hertz  = concatenate_df("03987", start_weather, end_weather)
weather_hist_DE_AMPRION  = concatenate_df("4336", start_weather, end_weather)
weather_hist_DE_TENNET   = concatenate_df("3668", start_weather, end_weather)
weather_hist_DE_TRANSNET = concatenate_df("2712", start_weather, end_weather)

# convert timezone to Europe/Berlin (aligned with generation data)
weather_hist_DE_50Hertz  = weather_hist_DE_50Hertz.tz_convert("Europe/Berlin")
weather_hist_DE_AMPRION  = weather_hist_DE_AMPRION.tz_convert("Europe/Berlin")
weather_hist_DE_TENNET   = weather_hist_DE_TENNET.tz_convert("Europe/Berlin")
weather_hist_DE_TRANSNET = weather_hist_DE_TRANSNET.tz_convert("Europe/Berlin")

radiation_global


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

sunshine_duration
temperature_air_mean_200
temperature_dew_point_mean_200
wind_direction
wind_speed
radiation_global


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

sunshine_duration
temperature_air_mean_200
temperature_dew_point_mean_200
wind_direction
wind_speed
radiation_global


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

sunshine_duration
temperature_air_mean_200
temperature_dew_point_mean_200
wind_direction
wind_speed
radiation_global


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

sunshine_duration
temperature_air_mean_200
temperature_dew_point_mean_200
wind_direction
wind_speed


In [20]:
#save weather data
weather_hist_DE_50Hertz.to_parquet("data/weather_hist_DE_50Hertz.parquet")
weather_hist_DE_AMPRION.to_parquet("data/weather_hist_DE_AMPRION.parquet") 
weather_hist_DE_TENNET.to_parquet("data/weather_hist_DE_TENNET.parquet")   
weather_hist_DE_TRANSNET.to_parquet("data/weather_hist_DE_TRANSNET.parquet")

